In [1]:
import cv2
import os

# Path to the video file
video_path = 'vdo.mp4'

# Create an output directory to save frames if it doesn't already exist
output_dir = 'extracted_frames'
os.makedirs(output_dir, exist_ok=True)

# Open the video file
cap = cv2.VideoCapture(video_path)

# Frame extraction
frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    
    # Break the loop if there are no frames left to read
    if not ret:
        break

    # Save the frame as an image file
    frame_filename = os.path.join(output_dir, f'frame_{frame_count:04d}.jpg')
    cv2.imwrite(frame_filename, frame)
    
    # Print progress
    print(f"Extracted {frame_filename}")
    
    # Increment frame count
    frame_count += 1

# Release the video capture object
cap.release()
print("Frame extraction complete.")


Extracted extracted_frames\frame_0000.jpg
Extracted extracted_frames\frame_0001.jpg
Extracted extracted_frames\frame_0002.jpg
Extracted extracted_frames\frame_0003.jpg
Extracted extracted_frames\frame_0004.jpg
Extracted extracted_frames\frame_0005.jpg
Extracted extracted_frames\frame_0006.jpg
Extracted extracted_frames\frame_0007.jpg
Extracted extracted_frames\frame_0008.jpg
Extracted extracted_frames\frame_0009.jpg
Extracted extracted_frames\frame_0010.jpg
Extracted extracted_frames\frame_0011.jpg
Extracted extracted_frames\frame_0012.jpg
Extracted extracted_frames\frame_0013.jpg
Extracted extracted_frames\frame_0014.jpg
Extracted extracted_frames\frame_0015.jpg
Extracted extracted_frames\frame_0016.jpg
Extracted extracted_frames\frame_0017.jpg
Extracted extracted_frames\frame_0018.jpg
Extracted extracted_frames\frame_0019.jpg
Extracted extracted_frames\frame_0020.jpg
Extracted extracted_frames\frame_0021.jpg
Extracted extracted_frames\frame_0022.jpg
Extracted extracted_frames\frame_0

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models

# 1. Load the dataset
(ds_train, ds_test), ds_info = tfds.load('horses_or_humans', split=['train', 'test'], with_info=True, as_supervised=True)

# 2. View the number of testing and training images
train_size = ds_info.splits['train'].num_examples
test_size = ds_info.splits['test'].num_examples
print(f"Number of training images: {train_size}")
print(f"Number of testing images: {test_size}")

# 3. Plot some images
def plot_samples(dataset, num_images=9):
    plt.figure(figsize=(10, 10))
    for i, (image, label) in enumerate(dataset.take(num_images)):
        plt.subplot(3, 3, i + 1)
        plt.imshow(image)
        plt.axis('off')
        plt.title("Horse" if label == 0 else "Human")
    plt.show()

# Display first 9 images from the training dataset
plot_samples(ds_train)

# 4. Normalizing the training data and preprocessing the dataset
def preprocess_data(dataset, batch_size=32):
    def normalize_image(image, label):
        image = tf.image.resize(image, (128, 128))  # Resize images to 128x128
        image = tf.cast(image, tf.float32) / 255.0  # Normalize pixel values to [0, 1]
        return image, label

    return dataset.map(normalize_image).batch(batch_size).prefetch(tf.data.AUTOTUNE)

train_ds = preprocess_data(ds_train)
test_ds = preprocess_data(ds_test)

# 5. Build a Convolutional Neural Network (ResNet) for classification
base_model = tf.keras.applications.ResNet50(input_shape=(128, 128, 3), include_top=False, weights='imagenet')
base_model.trainable = False  # Freeze the ResNet layers

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Binary classification (horse or human)
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 6. Train the CNN model and show the accuracy
history = model.fit(train_ds, epochs=5, validation_data=test_ds)

# Plot the training and validation accuracy
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.show()

# 7. Show the testing accuracy
test_loss, test_acc = model.evaluate(test_ds)
print(f"Test accuracy: {test_acc}")